In [2]:
import os

02_feature_engineering_pipeline

Objective:
    1. Reading the file from the loacation
    2. Doing data preprcessing like extracting relevant features, maping values.
    3. Let's do 1 and 2

feature_engineering: \
  root_dir: artifacts/feature_engineering  \
  training_data_path: artifacts/feature_engineering  \
  training_data_file: artifacts/feature_engineering/tweets.cs


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    training_data_path: Path
    training_data_file: str
    training_cleansed_data: str
    

In [4]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT\\research'

In [5]:
import os
os.chdir('..')

In [6]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT'

In [7]:
from pathlib import Path

CONFIG_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/config/config.yaml")
PARAMS_FILE_PATH = Path("c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT/params.yaml")

In [8]:
from src.textClassifier.constants import *
from src.textClassifier.utils.commons import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.feature_engineering

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            training_data_path=config.training_data_path,
            training_data_file=config.training_data_file,
            training_cleansed_data=config.training_cleansed_data
        )

        return data_preprocessing_config

In [37]:
# from textClassifier import logger
# import zipfile
# import os
# from textClassifier.utils.commons import get_size
# import gdown
# import spacy
# from nltk.corpus import stopwords
# import string



# class DataPreprocessing:
#     def __init__(self, config: DataPreprocessingConfig):
#         self.config = config

#     def preprocess_text(self):
#         nlp = spacy.load('en_core_web_sm')
#         stop_words = set(stopwords.words('english'))
#         text = text.lower() # Convert to lowercase

#         # Remove punctuation
#         text = text.translate(str.maketrans('', '', string.punctuation))

#         # Tokenize and lemmatize
#         doc = nlp(text)
#         tokens = [token.lemma_ for token in doc if token.text not in stop_words]
#         return ' '.join(tokens)

#     def pass_data_to_text_process(self):
#         # self.config.training_data_file
#         data['cleaned_text'] = data['text'].apply(preprocess_text)
#         return data


In [38]:
# dp = DataPreprocessing(config_manager)

C:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\artifacts\data_ingestion\Tweets.csv

In [39]:
# dp.preprocess_text('hi dude Happy was how are you doing')

'hi dude happy'

In [10]:
from textClassifier import logger
import zipfile
import os
from textClassifier.utils.commons import get_size
import gdown
import spacy
from nltk.corpus import stopwords
import string
import pandas as pd


class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')
        self.stop_words = set(stopwords.words('english'))


    def text_process(self, text: str) -> str:
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]
        return ' '.join(tokens)
    
    def get_data_from_features(self) -> pd.DataFrame:
        data = pd.read_csv(self.config.training_data_file)
        data['cleaned_text'] = data['text'].apply(self.text_process)
        return data
    
    def save_data(self, data: pd.DataFrame, path_name: str='cleaned_tweets.csv') -> None:
        out_path_name = Path(self.config.training_cleansed_data)
        data.to_csv(out_path_name, index=False)

        





        

In [80]:
config_manager = ConfigurationManager()
get_data_pre_config = config_manager.get_data_preprocessing_config()
data_processing = DataPreprocessing(config = get_data_pre_config)
output = data_processing.get_data_from_features()
save_data = data_processing.save_data()

[2025-02-08 15:30:36,292: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-08 15:30:36,297: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-08 15:30:36,299: INFO: commons: created directory at: artifacts]
[2025-02-08 15:30:36,301: INFO: commons: created directory at: artifacts/feature_engineering]


TypeError: save_data() missing 1 required positional argument: 'data'

In [79]:
print(output.head())

             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [37]:
from textClassifier import logger
import pandas as pd
import spacy
from nltk.corpus import stopwords
import string
from pathlib import Path
import torch
from transformers import BertTokenizer

class DataPreprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config
        self.nlp = spacy.load('en_core_web_sm')  # Load spaCy model
        self.stop_words = set(stopwords.words('english'))  # Load stopwords
        self.mapping_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def text_process(self, text: str) -> str:
        """
        Preprocesses the input text by:
        1. Converting to lowercase
        2. Removing punctuation
        3. Tokenizing and lemmatizing
        4. Removing stopwords
        """
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize and lemmatize
        doc = self.nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in self.stop_words]

        return ' '.join(tokens)

    def get_data_from_features(self) -> pd.DataFrame:
        """
        Loads the dataset from the path specified in the config and applies preprocessing to the text column.
        """
        # Check if the file exists
        if not Path(self.config.training_data_file).exists():
            raise FileNotFoundError(f"File not found: {self.config.training_data_file}")

        # Load the dataset
        data = pd.read_csv(self.config.training_data_file)

        # Apply preprocessing to the text column
        data['cleaned_text'] = data['text'].apply(self.text_process)

        return data

    def save_data(self, data: pd.DataFrame, path_name: str = 'cleaned_tweets.csv') -> None:
        """
        Saves the preprocessed data to the specified output file.

        Args:
            data (pd.DataFrame): The preprocessed DataFrame to save.
            path_name (str): The name of the output file. Defaults to "cleaned_tweets.csv".
        """
        # Create the full output file path
        out_path_name = Path(self.config.training_cleansed_data) / path_name

        # Save the DataFrame to CSV
        data.to_csv(out_path_name, index=False)
        logger.info(f"Preprocessed data saved to {out_path_name}")


    def mapping_labels_func(self, data: pd.DataFrame) -> pd.DataFrame:

        data['label'] = data['airline_sentiment'].map(self.mapping_labels)
        logger.info(f"values are mapped in the {data}")
        return data
    
    def tokenizing_func(self, data: pd.DataFrame, text_column: str = 'cleaned_text') -> dict:
        # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # return tokenizer(data, padding='max_length', truncation=True, max_length=128)
    
        tokenized_inputs = self.tokenizer(
            data[text_column].tolist(),  # Convert text column to list
            padding='max_length',        # Pad to max length
            truncation=True,             # Truncate to max length
            max_length=128,              # Set max length
            return_tensors='pt'          # Return PyTorch tensors
        )
        return tokenized_inputs




from sklearn.model_selection import train_test_split
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        # self.tokenized_text = tokenized_text

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def train_test_split_func(self, data: pd.DataFrame) -> pd.DataFrame:
        train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['tokenized'].to_list(), data['label'].to_list(), test_size=0.3, random_state=42)
        val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)
        return train_texts, temp_texts, train_labels, temp_labels, val_texts, test_texts, val_labels, test_labels
    
    def convert_to_dict(self, tokenized_text) -> dict:
        input_ids = [d['input_ids'] for d in self.tokenized_texts]
        attention_masks = [d['attention_mask'] for d in self.tokenized_texts]
        return {'input_ids': input_ids, 'attention_mask': attention_masks}



In [ ]:
# Map labels to integers
# label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
# data['label'] = data['airline_sentiment'].map(label_mapping)

# from transformers import BertTokenizer

# # Tokenization
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize the data
# def tokenize_function(text):
#     return tokenizer(text, padding='max_length', truncation=True, max_length=128)

# data['tokenized'] = data['cleaned_text'].apply(tokenize_function)

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the dataset
# train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['tokenized'].to_list(), data['label'].to_list(), test_size=0.3, random_state=42)
# val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

In [36]:
# Example usage
if __name__ == "__main__":
    # Assuming ConfigurationManager is defined elsewhere
    config_manager = ConfigurationManager()

    # Get the data preprocessing config
    get_data_pre_config = config_manager.get_data_preprocessing_config()

    # Initialize DataPreprocessing with the config
    data_processing = DataPreprocessing(config=get_data_pre_config)

    # Load and preprocess the data
    output = data_processing.get_data_from_features()

    # Save the preprocessed data
    data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

    # Map labels to numerical values
    output = data_processing.mapping_labels_func(data=output)
    print(output.head())

    # Tokenize the data
    tokenized_data = data_processing.tokenizing_func(data=output)
    print(tokenized_data[:5])

    # Split the data into train, validation, and test sets
    sentiment_dataset = SentimentDataset(encodings=tokenized_data, labels=output['label'].tolist())
    train_texts, val_texts, test_texts, train_labels, val_labels, test_labels = sentiment_dataset.train_test_split_func(output)

    # Convert tokenized texts to dictionary format
    tokenized_dict = sentiment_dataset.convert_to_dict(tokenized_data)
    print(tokenized_dict[:5])


[2025-02-13 15:39:49,751: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-13 15:39:49,757: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-13 15:39:49,759: INFO: commons: created directory at: artifacts]
[2025-02-13 15:39:49,761: INFO: commons: created directory at: artifacts/feature_engineering]


In [30]:
config_manager = ConfigurationManager()

# Get the data preprocessing config
get_data_pre_config = config_manager.get_data_preprocessing_config()

# Initialize DataPreprocessing with the config
data_processing = DataPreprocessing(config=get_data_pre_config)

# Load and preprocess the data
output = data_processing.get_data_from_features()

# Save the preprocessed data
data_processing.save_data(data=output, path_name='cleaned_tweets.csv')

# mapping = data_processing.mapping_labels_func()
output = data_processing.mapping_labels_func(data = output)
print(output.head())
tokenized_data = data_processing.tokenizing_func(data=output)
print(tokenized_data[:5])



[2025-02-12 15:56:11,364: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\config\config.yaml loaded successfully]
[2025-02-12 15:56:11,368: INFO: commons: yaml file: c:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\params.yaml loaded successfully]
[2025-02-12 15:56:11,370: INFO: commons: created directory at: artifacts]
[2025-02-12 15:56:11,373: INFO: commons: created directory at: artifacts/feature_engineering]
[2025-02-12 15:58:20,522: INFO: 751830446: Preprocessed data saved to artifacts\feature_engineering\cleaned_tweets.csv]
[2025-02-12 15:58:20,533: INFO: 751830446: values are mapped in the                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   